# Preparing Data

In [8]:
import pandas as pd

dataset = pd.read_csv('housing.csv')

In [9]:
print(dataset.shape)
dataset[:5]

(506, 13)


,crim,zn,indus,chas,nox,age,rm,dis,rad,tax,ptratio,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,5.33,36.2


In [10]:
# Move 'medv' target attribute column to front,
dataset = pd.concat([dataset['medv'], dataset.drop(['medv'], axis=1)], axis=1)

In [11]:
from sklearn.model_selection import train_test_split
training_dataset, validation_dataset = train_test_split(dataset, test_size=0.1)

print(training_dataset.shape)
print(validation_dataset.shape)

(455, 13)
(51, 13)


In [12]:
training_dataset.to_csv('training_dataset.csv', index=False, header=False)
validation_dataset.to_csv('validation_dataset.csv', index=False, header=False)

In [13]:
import sagemaker

print(sagemaker.__version__)

sess = sagemaker.Session()
bucket = sess.default_bucket()

prefix = 'adeelml-boston-housing'
training_data_path = sess.upload_data(path='training_dataset.csv', key_prefix=prefix + '/input/training')
validation_data_path = sess.upload_data(path='validation_dataset.csv', key_prefix=prefix + '/input/validation')

print(training_data_path)
print(validation_data_path)

2.41.0
s3://sagemaker-us-west-1-886035371869/adeelml-boston-housing/input/training/training_dataset.csv
s3://sagemaker-us-west-1-886035371869/adeelml-boston-housing/input/validation/validation_dataset.csv


# Configuring a training job

In [14]:
# estimator let you select the appropriate algorithm
import boto3
from sagemaker import image_uris

# with these apis, we can easily find the name of the algorithm. Sagemaker algorithm are packaged in dockers.
region = boto3.Session().region_name    
container = image_uris.retrieve('linear-learner', region)

print(container)

from sagemaker.estimator import Estimator

role = sagemaker.get_execution_role() 

ll_estimator = Estimator(container,
    role=role, 
    instance_count=1,
    instance_type='ml.m5.large',
    output_path='s3://{}/{}/output'.format(bucket, prefix)
)

ll_estimator.set_hyperparameters(predictor_type='regressor', mini_batch_size=32)

632365934929.dkr.ecr.us-west-1.amazonaws.com/linear-learner:1


In [15]:
# data channel is a named source of data passed to a SageMaker estimator.
# The TrainingInput() API lets us define their location, their format, whether they are compressed
training_data_channel   = sagemaker.TrainingInput(s3_data=training_data_path, content_type='text/csv')
validation_data_channel = sagemaker.TrainingInput(s3_data=validation_data_path, content_type='text/csv')
# ll-estimator takes chanel path
ll_data = {'train': training_data_channel, 'validation': validation_data_channel}

# Launching a training job

In [16]:
ll_estimator.fit(ll_data)

2021-06-17 18:03:38 Starting - Starting the training job...
2021-06-17 18:03:40 Starting - Launching requested ML instancesProfilerReport-1623953018: InProgress
......
2021-06-17 18:05:04 Starting - Preparing the instances for training......
2021-06-17 18:06:06 Downloading - Downloading input data
2021-06-17 18:06:06 Training - Downloading the training image...
2021-06-17 18:06:33 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
[06/17/2021 18:06:36 INFO 139681104275264] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'un

In [17]:
%%bash -s "$ll_estimator.output_path"
aws s3 ls --recursive $1

2021-06-17 18:06:49        920 adeelml-boston-housing/output/linear-learner-2021-06-17-18-03-38-444/output/model.tar.gz
2021-06-17 18:06:52          0 adeelml-boston-housing/output/linear-learner-2021-06-17-18-03-38-444/profiler-output/framework/training_job_end.ts
2021-06-17 18:06:47      39971 adeelml-boston-housing/output/linear-learner-2021-06-17-18-03-38-444/profiler-output/system/incremental/2021061718/1623953100.algo-1.json
2021-06-17 18:06:47     140784 adeelml-boston-housing/output/linear-learner-2021-06-17-18-03-38-444/profiler-output/system/incremental/2021061718/1623953160.algo-1.json
2021-06-17 18:06:52          0 adeelml-boston-housing/output/linear-learner-2021-06-17-18-03-38-444/profiler-output/system/training_job_end.ts
2021-06-17 18:08:40     322342 adeelml-boston-housing/output/linear-learner-2021-06-17-18-03-38-444/rule-output/ProfilerReport-1623953018/profiler-output/profiler-report.html
2021-06-17 18:08:40     168688 adeelml-boston-housing/output/linear-learner-20

# Deploying a model

In [18]:
from time import strftime, gmtime
timestamp = strftime('%d-%H-%M-%S', gmtime())

endpoint_name = 'adeelml-linear-learner-demo-'+timestamp
print(endpoint_name)

adeelml-linear-learner-demo-17-18-18-47


In [19]:
ll_predictor = ll_estimator.deploy(endpoint_name=endpoint_name, 
                        initial_instance_count=1, 
                        instance_type='ml.t2.medium')

-----------------!

In [20]:
test_sample = '0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,4.98'

In [25]:
# ll_predictor.content_type = 'text/csv'
ll_predictor.serializer = sagemaker.serializers.CSVSerializer()
ll_predictor.deserializer = sagemaker.deserializers.CSVDeserializer()

response = ll_predictor.predict(test_sample)
print(response)

[['30.39902114868164']]


In [26]:
test_samples = ['0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,4.98',
                '0.02731,0.00,7.070,0,0.4690,6.4210,78.90,4.9671,2,242.0,17.80,9.14']

response = ll_predictor.predict(test_samples)
print(response)

[['30.39902114868164'], ['24.998929977416992']]


In [27]:
runtime = boto3.Session().client(service_name='runtime.sagemaker') 

response = runtime.invoke_endpoint(EndpointName=endpoint_name, 
                                  ContentType='text/csv', 
                                  Body=test_sample)

print(response['Body'].read())

b'{"predictions": [{"score": 30.39902114868164}]}'


In [28]:
ll_predictor.delete_endpoint()